# Scipy 1.9.1 EBM-only test

We are pretty confident scipy is the issue: but does it make a difference if you call the EBM not from FaIR?

In [ ]:
import os

import matplotlib.pyplot as pl
import numpy as np
import pandas as pd

from fair.energy_balance_model import EnergyBalanceModel

In [ ]:
df_forcing = pd.read_csv(
    os.path.join("..", "data", "forcing", "RFMIP-ERF-tier2.csv")
)

In [ ]:
df_cr = pd.read_csv(os.path.join("..", "data", "calibration", "4xCO2_cummins_ebm3.csv"))

cr = {}
                    
for irow, row in df_cr.iterrows():
    cr[row.model, row.run] = EnergyBalanceModel(
        ocean_heat_capacity=[row.C1, row.C2, row.C3],
        ocean_heat_transfer=[row.kappa1, row.kappa2, row.kappa3],
        deep_ocean_efficacy=row.epsilon,
        gamma_autocorrelation=row.gamma,
        sigma_xi=row.sigma_xi,
        sigma_eta=row.sigma_eta,
        forcing_4co2=row.F_4xCO2,
        stochastic_run=True,
        seed=16
    )
    cr[row.model, row.run].add_forcing(forcing = df_forcing['GISS-E2-1-G TOT'].values, timestep=1)

In [ ]:
for irow, row in df_cr.iterrows():
    cr[row.model, row.run].run()

In [ ]:
for irow, row in df_cr.iterrows():
    pl.plot(cr[row.model, row.run].temperature[:,0])

In [ ]:
for irow, row in df_cr.iterrows():
    pl.plot(cr[row.model, row.run].temperature[:,1])

In [ ]:
for irow, row in df_cr.iterrows():
    pl.plot(cr[row.model, row.run].temperature[:,2])

In [ ]:
cr[row.model, row.run].temperature.shape

Well, I suppose the good news, if there is good news, is that it isn't FaIR causing the issues.

In [ ]:
# these are the culprits!

for irow, row in df_cr.iterrows():
    if max(cr[row.model, row.run].temperature[:,0])>10:
        print(row.model, row.run)

In [ ]:
cr_determ = {}

for irow, row in df_cr.iterrows():
    cr_determ[row.model, row.run] = EnergyBalanceModel(
        ocean_heat_capacity=[row.C1, row.C2, row.C3],
        ocean_heat_transfer=[row.kappa1, row.kappa2, row.kappa3],
        deep_ocean_efficacy=row.epsilon,
        gamma_autocorrelation=row.gamma,
        sigma_xi=row.sigma_xi,
        sigma_eta=row.sigma_eta,
        forcing_4co2=row.F_4xCO2,
        stochastic_run=False,
        seed=16
    )
    cr_determ[row.model, row.run].add_forcing(forcing = df_forcing['GISS-E2-1-G TOT'].values, timestep=1)

In [ ]:
for irow, row in df_cr.iterrows():
    cr_determ[row.model, row.run].run()

In [ ]:
for irow, row in df_cr.iterrows():
    pl.plot(cr_determ[row.model, row.run].temperature[:,0])

We can see the deterministic model is behaving. So the issue is entirely in the stochastic model.

In [ ]:
cr[('CAMS-CSM1-0', 'r1i1p1f1')].stochastic_d

In [ ]:
cr[('CAMS-CSM1-0', 'r1i1p1f1')].eb_matrix_d

In [ ]:
cr[('UKESM1-0-LL', 'r1i1p1f2')].eb_matrix_d

In [ ]:
for irow, row in df_cr.iterrows():
    print(row.model, row.run, cr[row.model, row.run].eb_matrix_d[0,0])

In [ ]:
cr[('CAMS-CSM1-0', 'r1i1p1f1')].ocean_heat_capacity

In [ ]:
cr[('CAMS-CSM1-0', 'r1i1p1f1')].ocean_heat_transfer

In [ ]:
cr[('CAMS-CSM1-0', 'r1i1p1f1')].deep_ocean_efficacy